In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import json

In [ ]:
df = pd.read_pickle("citibikedata/9000timeslots.pickle")
# Please note that some unused columns were deleted.
# However, in all other respects the pickled data was "raw".
# Row cleaning had not yet taken place.

In [ ]:
STATIONDB = pd.read_pickle("citibikedata/ALLSTATIONS.pickle")

In [ ]:
TARGETSTATION = 465   # Happens to be my favorite TimesSq station on 41st.
#TARGETSTATION = 523   # This is a station in prime territory but strangely marked as low-activity

In [ ]:
# Houston we have a problem!
# We are getting files with "last_reported" of ZERO so those must be filtered out of the dataframe first.
df = df[df['last_reported'] > 1000]
# OK now the df is clean of bad timestamps in the "last_reported" column.
# Converting from typical second-granularity epoch timestamp requires unit='s'
df['most_recent_conn_DT'] = pd.to_datetime(df['last_reported'], unit='s')

In [ ]:
# BRING IN THE STATION PARTICULARS (lat, long, station name)
# This join should not be done more than once; re-join fails.
# WHOA do not do the join too early!  We have millions of rows at this point.
#df = df.join(STATIONDB)

In [ ]:
# The shape of a dataframe is its row count x column count
df.dtypes

In [ ]:
# if you want a successful plotting of just one station:
#df[df['station_id']==TARGETSTATION].plot(x='most_recent_conn_DT', y=['num_bikes_available'])
df[df['station_id']==TARGETSTATION]

# 1: STATIONS WITH MOST "volatility"

Every station sends its reports to HQ only sporadically, not on a fixed schedule.

So as a quick measure of volatility of station S, we could take the time-sorted signatures for station S, and determine the velocity between each adjacent pair S[i] and S[i+1], and compute the sum of the velocities.

The velocity could simply be the sum, across all columns C, of abs(S[i][c] - S[i+1][c]).  The will as desired produce a velocity of zero if two adjacent reports actually had no net change to report.

We could mute the velocity by the duration between S[i] and S[i+1] but this isn't really necessary since max(i) itself will be higher for the highest-active stations anyway, so they will naturally have more velocities being summed.

## 1.1:  "QuickCompute Volatility"

The quickest approach to computing volatility would be to simply produce the count of individual report rows per station.  There is already a great deal
of spread on that particular metric.

Let's compute a histogram based on QC volatility!


In [ ]:
df = df.drop(columns=['is_installed','is_renting','is_returning','ts'], 
             errors='ignore')

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

In [ ]:
df.sort_values(by=['station_id','most_recent_conn_DT'], inplace=True)

In [ ]:
# This will create an obj of type DataFrameGroupBy
per_station_info = df.groupby('station_id')

In [ ]:
# OK so there are 845 actual listed stations in the official station DB.
# But not all are online as you can see here:
per_station_info.ngroups

In [ ]:
type(per_station_info)

In [ ]:
# TRIVIAL  "QuickVolatility" is just the count per station.
# Let's create a dataframe that can rankorder all the stations?
df_station_to_quickvol = per_station_info.count().join(STATIONDB)
# we now have a dataframe, and we have the station names/geo info in there via that join.
df_station_to_quickvol.hist(column='last_reported',bins=10)

In [ ]:
df_station_to_quickvol__most_active = df_station_to_quickvol.sort_values(by='last_reported', ascending=False)[:20]

In [ ]:
topstation = df_station_to_quickvol__most_active.iloc[0]
topstation

# The Top 20 Most-Active Stations

In [ ]:
df_station_to_quickvol__most_active['name']

In [ ]:
# For GMaps to function, Google requires you obtain and enable an API key:
#     https://developers.google.com/maps/documentation/javascript/get-api-keyfrom credentials import GOOMAPSAPIKEY

In [ ]:
from bokeh.io import output_file, show, reset_output
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap

reset_output()
output_file("nyc_gmaps_2.html")

map_options = GMapOptions(lat=topstation['lat'],
                          lng=topstation['lon'], map_type="roadmap", zoom=11)

p = gmap(GOOMAPSAPIKEY, map_options, title="Most Active CitiBike Stations")

source = ColumnDataSource(
    data=dict(lat=df_station_to_quickvol__most_active['lat'],
              lon=df_station_to_quickvol__most_active['lon'])
)

p.circle(x="lon", y="lat", size=10, fill_color="blue", fill_alpha=0.8, source=source)

show(p)

# The Least-Active Stations

In [ ]:
num_stations = len(df_station_to_quickvol)
df_station_to_quickvol__bottomfeeders = df_station_to_quickvol.sort_values(by='last_reported', ascending=False)[num_stations-20:num_stations-1]

In [ ]:
df_station_to_quickvol__bottomfeeders

In [ ]:
from bokeh.io import output_file, show, reset_output
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap

reset_output()
output_file("nyc_least_active.html")

map_options = GMapOptions(lat=topstation['lat'],
                          lng=topstation['lon'], map_type="roadmap", zoom=11)

p = gmap(GOOMAPSAPIKEY, map_options, title="Least Active CitiBike Stations")

source = ColumnDataSource(
    data=dict(lat=df_station_to_quickvol__bottomfeeders['lat'],
              lon=df_station_to_quickvol__bottomfeeders['lon'])
)

p.circle(x="lon", y="lat", size=10, fill_color="red", fill_alpha=0.8, source=source)

show(p)

## Oddness

I have a weird feeling about these stations I've marked as inactive.  Many are in prime areas of Manhattan.

So I want to do more research and have chosen to look at station 523.
